In [2]:
import requests
import pandas as pd
import numpy as np
import json

In [3]:
df=pd.DataFrame(columns=['date','net_asset_value','id'])

In [4]:
url='https://fintual.cl/api/real_assets/15077/days?from_date=%222022-01-01%22&to_date=%222022-12-21%22'

response=requests.get(url).json()

funds=["15077","186","187","188"]

for fund in funds:
    url='https://fintual.cl/api/real_assets/'+fund+'/days?from_date=%222022-01-01%22&to_date=%222022-12-21%22'
    response=requests.get(url).json()
    for date_fund in response['data']:
        net_asset_value=date_fund['attributes']['net_asset_value']
        date=date_fund['attributes']['date']
        id=date_fund['id']
        df= df.append({'date':date,'net_asset_value':net_asset_value,'id':id},ignore_index=True)


In [5]:
df['fund_code']=df['id'].str[:3].astype(int)

df['fund_name']=np.where(df['fund_code']==150, 'Very Conservative Streep',
                np.where(df['fund_code']==186, 'Risky Norris',
                np.where(df['fund_code']==187, 'Moderate Pit',
                np.where(df['fund_code']==188,'Conservative Clooney', 'Error'))))



In [16]:
df['net_asset_value_next_day'] = df.groupby('fund_name')['net_asset_value'].shift(1).astype(float)
df['net_asset_value']=df['net_asset_value'].astype(float)
df['profit(%)']=((df['net_asset_value']-df['net_asset_value_next_day'])/df['net_asset_value_next_day'])*100

In [17]:
df[['date','fund_name','profit(%)']]

,date,fund_name,profit(%)
0,2022-01-01,Very Conservative Streep,NaN
1,2022-01-02,Very Conservative Streep,0.011109
2,2022-01-03,Very Conservative Streep,0.011108
3,2022-01-04,Very Conservative Streep,0.011107
4,2022-01-05,Very Conservative Streep,0.011115
...,...,...,...
1415,2022-12-17,Conservative Clooney,-0.003254
1416,2022-12-18,Conservative Clooney,-0.003262
1417,2022-12-19,Conservative Clooney,-0.154449
1418,2022-12-20,Conservative Clooney,-0.164263


# intentar agregar INPUT